In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

% matplotlib inline

# 1. Import the training set and do basic EDA

In [2]:
#import the training set

In [3]:
df = pd.read_csv('train (2).csv')

In [4]:
submission = pd.read_csv('test.csv')

In [5]:
# some EDA

In [6]:
# df.shape

In [7]:
#df.isnull().sum().sort_values(ascending=False)[:26] #so these 26 variables have missing values

In [8]:
df[['SalePrice']].isnull().sum()

SalePrice    0
dtype: int64

In [9]:
#well, good news is that at least we have all the y values

In [10]:
#df[df.duplicated()] #no duplicates

## 2. Looking for strong correlations between price and other variables

In [11]:
# I am going to look for the variables with the highest corr (positive or negative) with SalePrice

In [12]:
df.corr()[['SalePrice']].apply(abs).sort_values('SalePrice', ascending=False)

,SalePrice
SalePrice,1.000000
Overall Qual,0.800207
Gr Liv Area,0.697038
Garage Area,0.650270
Garage Cars,0.648220
Total Bsmt SF,0.628925
1st Flr SF,0.618486
Year Built,0.571849
Year Remod/Add,0.550370
Full Bath,0.537969


In [13]:
df.corr()[['SalePrice']].apply(abs).sort_values('SalePrice', ascending=False).index

Index(['SalePrice', 'Overall Qual', 'Gr Liv Area', 'Garage Area',
       'Garage Cars', 'Total Bsmt SF', '1st Flr SF', 'Year Built',
       'Year Remod/Add', 'Full Bath', 'Garage Yr Blt', 'Mas Vnr Area',
       'TotRms AbvGrd', 'Fireplaces', 'BsmtFin SF 1', 'Lot Frontage',
       'Open Porch SF', 'Wood Deck SF', 'Lot Area', 'Bsmt Full Bath',
       'Half Bath', 'PID', '2nd Flr SF', 'Bsmt Unf SF', 'Bedroom AbvGr',
       'Enclosed Porch', 'Screen Porch', 'Kitchen AbvGr', 'Overall Cond',
       'MS SubClass', 'Id', '3Ssn Porch', 'Bsmt Half Bath', 'Low Qual Fin SF',
       'Mo Sold', 'Pool Area', 'BsmtFin SF 2', 'Yr Sold', 'Misc Val'],
      dtype='object')

In [14]:
# looked at the ones with corr over 30%, then out of those used all numeric and checked which ones make sense
# out of those that are categorical checked which ones have enough difference in values

In [15]:
# how about adding Cerntal Air, Kitchen Qual, PavedDrive
df['Paved Drive'].value_counts()

Y    1861
N     151
P      39
Name: Paved Drive, dtype: int64

## 3. Picking my predictors

In [16]:
# creating a new df with the top variables with the highest correlation with y
# doing the same for the submission set to match up

In [17]:
submission = submission[['Id','Overall Qual', 'Gr Liv Area', 'Garage Area',
       'Garage Cars', 'Total Bsmt SF', '1st Flr SF', 'Year Built',
       'Year Remod/Add', 'Full Bath', 'Garage Yr Blt','Mas Vnr Area',
       'TotRms AbvGrd', 'Kitchen Qual','Fireplaces', 'BsmtFin SF 1', 'Lot Frontage',
       'Open Porch SF', 'Wood Deck SF', 'Lot Area','Central Air','Paved Drive']]

In [18]:
newdf = df[['Id','SalePrice', 'Overall Qual', 'Gr Liv Area', 'Garage Area',
       'Garage Cars', 'Total Bsmt SF', '1st Flr SF', 'Year Built',
       'Year Remod/Add', 'Full Bath', 'Garage Yr Blt','Mas Vnr Area',
       'TotRms AbvGrd', 'Kitchen Qual','Fireplaces', 'BsmtFin SF 1', 'Lot Frontage',
       'Open Porch SF', 'Wood Deck SF', 'Lot Area','Central Air','Paved Drive']]

In [19]:
newdf.shape, submission.shape

((2051, 23), (879, 22))

In [20]:
#submission has one less column because it does not have SalePrice

In [21]:
#just filling missing values with zeros
#assuming the missing values are missing at the same rate fromt he training and test set it should not affect the model
#too much

newdf = newdf.fillna(0)
submission = submission.fillna(0)

In [22]:
#get dummies for the three categorical predictor variables
newdf = pd.get_dummies(newdf, columns=['Central Air','Kitchen Qual','Paved Drive'], drop_first=True)
submission = pd.get_dummies(submission, columns=['Central Air','Kitchen Qual','Paved Drive'], drop_first=True)

In [23]:
# I am a little paranoid about missing values
newdf = newdf.fillna(0)
submission = submission.fillna(0)

In [24]:
#these two variables for some reason gave off error, so I reassigned from floats to int

newdf["Garage Area"] = newdf["Garage Area"].astype(int)
newdf["Total Bsmt SF"] = newdf["Total Bsmt SF"].astype(int)

submission["Garage Area"] = submission["Garage Area"].astype(int)
submission["Total Bsmt SF"] = submission["Total Bsmt SF"].astype(int)

## 3. Add interractions

In [25]:
#some variables are highly correlated between themselved (garage area and garage cars), can try interractions
newdf['garage'] = newdf['Garage Area'] * newdf['Garage Cars']
newdf['Sq Ft'] = newdf['1st Flr SF'] * newdf['Total Bsmt SF']

submission['garage'] = submission['Garage Area'] * submission['Garage Cars']
submission['Sq Ft'] = submission['1st Flr SF'] * submission['Total Bsmt SF']

## 4. Create predictor and target variables. Standardize the predictors.

In [26]:
newdf.columns

Index(['Id', 'SalePrice', 'Overall Qual', 'Gr Liv Area', 'Garage Area',
       'Garage Cars', 'Total Bsmt SF', '1st Flr SF', 'Year Built',
       'Year Remod/Add', 'Full Bath', 'Garage Yr Blt', 'Mas Vnr Area',
       'TotRms AbvGrd', 'Fireplaces', 'BsmtFin SF 1', 'Lot Frontage',
       'Open Porch SF', 'Wood Deck SF', 'Lot Area', 'Central Air_Y',
       'Kitchen Qual_Fa', 'Kitchen Qual_Gd', 'Kitchen Qual_TA',
       'Paved Drive_P', 'Paved Drive_Y', 'garage', 'Sq Ft'],
      dtype='object')

In [27]:
features = ['Id','Overall Qual', 'Gr Liv Area', 'Garage Area',
       'Garage Cars', 'Total Bsmt SF', '1st Flr SF', 'Year Built',
       'Year Remod/Add', 'Full Bath', 'Garage Yr Blt', 'Mas Vnr Area',
       'TotRms AbvGrd', 'Fireplaces', 'BsmtFin SF 1', 'Lot Frontage',
       'Open Porch SF', 'Wood Deck SF', 'Lot Area', 'Central Air_Y',
       'Kitchen Qual_Fa', 'Kitchen Qual_Gd', 'Kitchen Qual_TA',
       'Paved Drive_P', 'Paved Drive_Y', 'garage', 'Sq Ft']

#all columns from my newdf except for SalePrice
    
X = newdf[features]
y = newdf['SalePrice']

In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = scaler.fit_transform(X)

#using the same feature list for the submission set to set X and scaling it

X_submission = submission[features]
X_submission = scaler.transform(X_submission)

##  5. TTS for the training set

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
print(X_train.shape)
print(X_test.shape)

(1538, 27)
(513, 27)


## 6. Finally creating my simple MLR and scoring it

In [30]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.intercept_) 
print(model.coef_)

0.8621025361868271
181551.05762720952
[   160.42477762  20580.971557    20789.68604804 -15612.58599389
 -13670.91444598  26332.43145848  10655.46307571   6926.00533633
   6998.86563432    455.0580014    6980.23750138   5036.52351063
   -137.91517397   4770.17283565   9247.00483347   1109.72198759
     79.93465596   1307.2350037    7748.79666688   -565.6949233
  -7884.23462386 -25267.37779968 -29275.44484881    374.55638562
   1289.63776226  32298.00649838 -36201.5792096 ]


In [31]:
# checking that the intercept is positive, score is 86% and coefficients make sense

In [32]:
#cross-validate
from sklearn.model_selection import cross_val_score
cross_val_score(model, X_train, y_train, cv = 10).mean()

0.8376768479647607

In [33]:
# cv score is not that far off model performance score
# try on the test set

In [34]:
model.score(X_test, y_test)

0.8754011139284391

In [55]:
#making the predictions on my training test set to make sure the predictions actually make sense (not negative, for ex)
model.predict(X_test)

array([124657.83797359, 233764.2025863 , 254367.39261074, 137446.72146039,
       197344.58755117, 356142.9112147 , 160137.82757228, 311736.50122965,
       171365.29778086,  81264.8676615 , 146448.02916319, 197410.09000769,
       164974.86513526, 105107.90282686, 115053.07566765,  88830.51842119,
       122484.81538231, 203201.40899428, 318984.54628359, 234744.60911307,
       203904.51598581, 200768.48955219, 151210.8844595 , 113697.18153046,
       157016.21134263, 237536.76572715, 168149.35330444, 121177.01740145,
       247234.82342365, 113923.46942071, 358714.57925481, 107283.99453105,
       120302.77111721, 182227.28136153, 198223.88439228,  79387.44304974,
       167173.57489099, 231983.24821569, 308283.5710916 , 111798.75256198,
       222992.10786668, 196391.3124267 , 292190.11720976, 183132.87943924,
        82881.94232308, 173539.89291091, 112472.27437398,  94254.81153454,
       293843.66021332, 141219.03807486, 374732.34038708,  63315.95270037,
       117642.33594604, 2

## 7. Applying the model to the kaggle test set

In [36]:
y_submission = model.predict(X_submission)

In [37]:
y_submission[0:5] #making sure that the results make sense

array([152491.21191136, 161327.4366651 , 205327.7427502 , 106147.05857414,
       179755.97594702])

## 8. Checking for column mismatch and fixing it

##### Credit goes to Ki-Hoon, who basically gave me the code.

In [38]:
[col for col in newdf.columns if col not in submission.columns]

['SalePrice']

In [39]:
[col for col in submission.columns if col not in newdf.columns]

['Kitchen Qual_Po']

In [40]:
all_cols = newdf.columns.union(submission.columns)

newdf = newdf.assign(**{col:0 for col in all_cols.difference(newdf.columns).tolist()})
submission = submission.assign(**{col:0 for col in all_cols.difference(submission.columns).tolist()})

#### Confirming that columns now match, and putting them in the same order
### Ben's code could be useful here...

In [41]:
newdf.shape, newdf.columns

((2051, 29),
 Index(['Id', 'SalePrice', 'Overall Qual', 'Gr Liv Area', 'Garage Area',
        'Garage Cars', 'Total Bsmt SF', '1st Flr SF', 'Year Built',
        'Year Remod/Add', 'Full Bath', 'Garage Yr Blt', 'Mas Vnr Area',
        'TotRms AbvGrd', 'Fireplaces', 'BsmtFin SF 1', 'Lot Frontage',
        'Open Porch SF', 'Wood Deck SF', 'Lot Area', 'Central Air_Y',
        'Kitchen Qual_Fa', 'Kitchen Qual_Gd', 'Kitchen Qual_TA',
        'Paved Drive_P', 'Paved Drive_Y', 'garage', 'Sq Ft', 'Kitchen Qual_Po'],
       dtype='object'))

In [42]:
submission.shape, submission.columns

((879, 29),
 Index(['Id', 'Overall Qual', 'Gr Liv Area', 'Garage Area', 'Garage Cars',
        'Total Bsmt SF', '1st Flr SF', 'Year Built', 'Year Remod/Add',
        'Full Bath', 'Garage Yr Blt', 'Mas Vnr Area', 'TotRms AbvGrd',
        'Fireplaces', 'BsmtFin SF 1', 'Lot Frontage', 'Open Porch SF',
        'Wood Deck SF', 'Lot Area', 'Central Air_Y', 'Kitchen Qual_Fa',
        'Kitchen Qual_Gd', 'Kitchen Qual_Po', 'Kitchen Qual_TA',
        'Paved Drive_P', 'Paved Drive_Y', 'garage', 'Sq Ft', 'SalePrice'],
       dtype='object'))

In [43]:
submission.shape

(879, 29)

In [44]:
submission = submission[newdf.columns]

In [45]:
assert (submission.columns == newdf.columns).all().all()

In [46]:
newdf.head(1)

,Id,SalePrice,Overall Qual,Gr Liv Area,Garage Area,Garage Cars,Total Bsmt SF,1st Flr SF,Year Built,Year Remod/Add,...,Lot Area,Central Air_Y,Kitchen Qual_Fa,Kitchen Qual_Gd,Kitchen Qual_TA,Paved Drive_P,Paved Drive_Y,garage,Sq Ft,Kitchen Qual_Po
0,109,130500,6,1479,475,2.0,725,725,1976,2005,...,13517,1,0,1,0,0,1,950.0,525625,0


In [47]:
submission.head(1)

,Id,SalePrice,Overall Qual,Gr Liv Area,Garage Area,Garage Cars,Total Bsmt SF,1st Flr SF,Year Built,Year Remod/Add,...,Lot Area,Central Air_Y,Kitchen Qual_Fa,Kitchen Qual_Gd,Kitchen Qual_TA,Paved Drive_P,Paved Drive_Y,garage,Sq Ft,Kitchen Qual_Po
0,2658,0,6,1928,440,1.0,1020,908,1910,1950,...,9142,0,1,0,0,0,1,440.0,926160,0


### 9. Adding the predicted SalesPrices to the submission set

In [48]:
submission['SalePrice'] = y_submission

### 10. Subsetting the two required columns. 

In [49]:
submission1 = submission[['Id','SalePrice']]

In [50]:
submission1.head(1)

,Id,SalePrice
0,2658,152491.211911


In [51]:
# I have the index column (so it's 3 columns instead of 2)
# I know there is a way to just drop index while saving to csv, but I couldn't figure out how
# so dropping it by hand

In [52]:
submission1.set_index('Id', inplace=True)

In [53]:
submission1.head(1)

,SalePrice
Id,
2658,152491.211911


In [54]:
#submission1.to_csv('submission3.csv')